<a href="https://colab.research.google.com/github/sayakpaul/EvoNorms-in-TensorFlow-2/blob/master/Mini_Inception_BN_ReLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Which GPU? 
!nvidia-smi

Sun Apr 19 03:34:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# TensorFlow imports
import tensorflow as tf
print(tf.__version__)

2.2.0-rc3


In [0]:
# Other imports
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import numpy as np

In [0]:
# Set the random seeds
tf.random.set_seed(666)
np.random.seed(666)

In [5]:
# Set up wandb for easy experiment tracking
!pip install wandb -q
import wandb
wandb.login()

     |████████████████████████████████| 1.4MB 4.6MB/s 
     |████████████████████████████████| 460kB 24.2MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
     |████████████████████████████████| 112kB 39.9MB/s 
     |████████████████████████████████| 102kB 12.4MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
# Load and preprocess CIFAR10 dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train / 255.
X_test = X_test / 255.
print(X_train.shape, X_test.shape)

170500096/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3) (10000, 32, 32, 3)


In [0]:
# Implementation comes from http://pyimg.co/mac01
def minigooglenet_functional(width, height, depth, classes):
	def conv_module(x, K, kX, kY, stride, chanDim, padding="same"):
		# define a CONV => BN => RELU pattern
		x = Conv2D(K, (kX, kY), strides=stride, padding=padding)(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = Activation("relu")(x)

		# return the block
		return x

	def inception_module(x, numK1x1, numK3x3, chanDim):
		# define two CONV modules, then concatenate across the
		# channel dimension
		conv_1x1 = conv_module(x, numK1x1, 1, 1, (1, 1), chanDim)
		conv_3x3 = conv_module(x, numK3x3, 3, 3, (1, 1), chanDim)
		x = concatenate([conv_1x1, conv_3x3], axis=chanDim)

		# return the block
		return x

	def downsample_module(x, K, chanDim):
		# define the CONV module and POOL, then concatenate
		# across the channel dimensions
		conv_3x3 = conv_module(x, K, 3, 3, (2, 2), chanDim,
			padding="valid")
		pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
		x = concatenate([conv_3x3, pool], axis=chanDim)

		# return the block
		return x

	# initialize the input shape to be "channels last" and the
	# channels dimension itself
	inputShape = (height, width, depth)
	chanDim = -1

	# define the model input and first CONV module
	inputs = Input(shape=inputShape)
	x = conv_module(inputs, 96, 3, 3, (1, 1), chanDim)

	# two Inception modules followed by a downsample module
	x = inception_module(x, 32, 32, chanDim)
	x = inception_module(x, 32, 48, chanDim)
	x = downsample_module(x, 80, chanDim)

	# four Inception modules followed by a downsample module
	x = inception_module(x, 112, 48, chanDim)
	x = inception_module(x, 96, 64, chanDim)
	x = inception_module(x, 80, 80, chanDim)
	x = inception_module(x, 48, 96, chanDim)
	x = downsample_module(x, 96, chanDim)

	# two Inception modules followed by global POOL and dropout
	x = inception_module(x, 176, 160, chanDim)
	x = inception_module(x, 176, 160, chanDim)
	x = AveragePooling2D((7, 7))(x)
	x = Dropout(0.5)(x)

	# softmax classifier
	x = Flatten()(x)
	x = Dense(classes)(x)
	x = Activation("softmax")(x)

	# create the model
	model = Model(inputs, x, name="minigooglenet")

	# return the constructed network architecture
	return model

In [0]:
# Hyperparams
BATCH_SIZE = 128
EPOCHS = 60

In [0]:
# Import wandb's Keras callback
from wandb.keras import WandbCallback

In [10]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="bn-relu-adam-no-aug")

# Compile and train this model
model = minigooglenet_functional(32, 32, 3, 10)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
	metrics=["accuracy"])
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[WandbCallback()])

Epoch 1/60
391/391 [==============================] - 19s 49ms/step - loss: 1.2531 - accuracy: 0.5490 - val_loss: 1.5977 - val_accuracy: 0.4144
Epoch 2/60
391/391 [==============================] - 18s 46ms/step - loss: 0.7932 - accuracy: 0.7216 - val_loss: 1.1352 - val_accuracy: 0.6352
Epoch 3/60
391/391 [==============================] - 18s 46ms/step - loss: 0.6114 - accuracy: 0.7885 - val_loss: 0.7932 - val_accuracy: 0.7158
Epoch 4/60
391/391 [==============================] - 18s 46ms/step - loss: 0.4920 - accuracy: 0.8302 - val_loss: 0.8799 - val_accuracy: 0.7032
Epoch 5/60
391/391 [==============================] - 18s 46ms/step - loss: 0.4119 - accuracy: 0.8583 - val_loss: 0.8826 - val_accuracy: 0.7272
Epoch 6/60
391/391 [==============================] - 18s 45ms/step - loss: 0.3385 - accuracy: 0.8828 - val_loss: 0.9387 - val_accuracy: 0.7149
Epoch 7/60
391/391 [==============================] - 18s 46ms/step - loss: 0.2816 - accuracy: 0.9023 - val_loss: 0.7375 - val_accuracy:

In [0]:
# Let's try one-hot encoding the labels and then see
y_train_ohe = tf.keras.utils.to_categorical(y_train)
y_test_ohe = tf.keras.utils.to_categorical(y_test)

In [12]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="bn-relu-adam-no-aug-ohe")

# Compile and train this model
model = minigooglenet_functional(32, 32, 3, 10)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
	metrics=["accuracy"])
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[WandbCallback()])

Epoch 1/60
391/391 [==============================] - 18s 47ms/step - loss: 1.2473 - accuracy: 0.5482 - val_loss: 2.0603 - val_accuracy: 0.3314
Epoch 2/60
391/391 [==============================] - 18s 46ms/step - loss: 0.7707 - accuracy: 0.7320 - val_loss: 1.2857 - val_accuracy: 0.5950
Epoch 3/60
391/391 [==============================] - 18s 46ms/step - loss: 0.5936 - accuracy: 0.7966 - val_loss: 1.2780 - val_accuracy: 0.5845
Epoch 4/60
391/391 [==============================] - 18s 46ms/step - loss: 0.4791 - accuracy: 0.8333 - val_loss: 0.9049 - val_accuracy: 0.7038
Epoch 5/60
391/391 [==============================] - 18s 46ms/step - loss: 0.4018 - accuracy: 0.8623 - val_loss: 1.0725 - val_accuracy: 0.6950
Epoch 6/60
391/391 [==============================] - 18s 46ms/step - loss: 0.3354 - accuracy: 0.8843 - val_loss: 0.8662 - val_accuracy: 0.7507
Epoch 7/60
391/391 [==============================] - 18s 46ms/step - loss: 0.2745 - accuracy: 0.9051 - val_loss: 0.9970 - val_accuracy:

In [13]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="bn-relu-sgd-no-aug-one-hot")

# Let's try changing the optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train this model
model = minigooglenet_functional(32, 32, 3, 10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(X_train, y_train_ohe,
                    validation_data=(X_test, y_test_ohe),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[WandbCallback()])

Epoch 1/60
391/391 [==============================] - 19s 48ms/step - loss: 1.4166 - accuracy: 0.4820 - val_loss: 1.8228 - val_accuracy: 0.3759
Epoch 2/60
391/391 [==============================] - 18s 47ms/step - loss: 0.9686 - accuracy: 0.6537 - val_loss: 1.2731 - val_accuracy: 0.5557
Epoch 3/60
391/391 [==============================] - 18s 47ms/step - loss: 0.7899 - accuracy: 0.7201 - val_loss: 0.9378 - val_accuracy: 0.6669
Epoch 4/60
391/391 [==============================] - 18s 46ms/step - loss: 0.6583 - accuracy: 0.7701 - val_loss: 0.9670 - val_accuracy: 0.6682
Epoch 5/60
391/391 [==============================] - 18s 47ms/step - loss: 0.5654 - accuracy: 0.8034 - val_loss: 0.9329 - val_accuracy: 0.6865
Epoch 6/60
391/391 [==============================] - 18s 46ms/step - loss: 0.4810 - accuracy: 0.8329 - val_loss: 1.0262 - val_accuracy: 0.6789
Epoch 7/60
391/391 [==============================] - 18s 47ms/step - loss: 0.4101 - accuracy: 0.8600 - val_loss: 0.9762 - val_accuracy:

In [0]:
# Let's try with data augmentation
aug = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=18, 
    zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, 
    shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

In [15]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="bn-relu-sgd-aug-ohe")

# Let's try changing the optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train this model
model = minigooglenet_functional(32, 32, 3, 10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(aug.flow(X_train, y_train_ohe),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
	validation_data=(X_test, y_test_ohe),
	validation_steps=X_test.shape[0] // BATCH_SIZE,
	epochs=EPOCHS,
	callbacks=[WandbCallback()])

Epoch 1/60
390/390 [==============================] - 10s 27ms/step - loss: 1.8879 - accuracy: 0.3054 - val_loss: 2.2616 - val_accuracy: 0.2069
Epoch 2/60
390/390 [==============================] - 10s 25ms/step - loss: 1.5720 - accuracy: 0.4274 - val_loss: 1.7422 - val_accuracy: 0.3926
Epoch 3/60
390/390 [==============================] - 10s 25ms/step - loss: 1.4400 - accuracy: 0.4759 - val_loss: 1.5125 - val_accuracy: 0.4727
Epoch 4/60
390/390 [==============================] - 10s 25ms/step - loss: 1.3493 - accuracy: 0.5150 - val_loss: 1.2899 - val_accuracy: 0.5461
Epoch 5/60
390/390 [==============================] - 10s 25ms/step - loss: 1.2495 - accuracy: 0.5569 - val_loss: 1.3680 - val_accuracy: 0.5353
Epoch 6/60
390/390 [==============================] - 10s 25ms/step - loss: 1.1991 - accuracy: 0.5744 - val_loss: 1.3477 - val_accuracy: 0.5596
Epoch 7/60
390/390 [==============================] - 10s 25ms/step - loss: 1.1538 - accuracy: 0.5859 - val_loss: 1.3506 - val_accuracy: